### Word Group In Youtube

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Portuguese"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 10000  # native word end index
unique_word_ratio = 85  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 85  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 25  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [7]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [8]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [9]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,que,22182513
1,a,22151153
2,o,21751123
3,não,17756980
4,de,15384404
...,...,...
278698,diagnosian,5
278699,aconchegarmo,5
278700,pumper,5
278701,patofa,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,que,22182513
1,a,22151153
2,o,21751123
3,não,17756980
4,de,15384404
...,...,...
19995,estimular,1133
19996,preferires,1133
19997,alistair,1133
19998,talismã,1133


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [15]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

19994

#### Youtube Sentence Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:09.740,00:00:11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA
1,00:00:52.520,00:00:53.500,é isso mesmo,ThC9MSlB1gA
2,00:01:22.480,00:01:25.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA
3,00:01:26.040,00:01:31.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA
4,00:01:31.380,00:01:37.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA
...,...,...,...,...
1370676,00:39:59.640,00:40:04.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o
1370677,00:40:04.680,00:40:12.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o
1370678,00:40:12.150,00:40:18.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o
1370679,00:40:18.990,00:40:25.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA
1,1,52.520,53.500,é isso mesmo,ThC9MSlB1gA
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA
...,...,...,...,...,...
1370676,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o
1370677,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o
1370678,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o
1370679,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o


In [19]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [20]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [21]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA,85.7,estevam,"olá, os, seguidores, do, à, todos"
1,1,52.520,53.500,é isso mesmo,ThC9MSlB1gA,100.0,,"mesmo, é, isso"
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA,75.0,"ascendência, portuguesa","tem, pode, temas, diferentes, são, quem"
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA,90.0,permanência,"perguntas, em, à, portugal, mas, você, muitas,..."
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA,83.3,"nacionalidade, portuguesa","pessoas, vivendo, em, se, portugal, podem, mui..."
...,...,...,...,...,...,...,...,...
1370676,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o,94.1,,"deixar, eu, mude, coisa, pode, nos, comentário..."
1370677,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,88.2,inscrevam,"receberem, canal, muito, no, se, para, ajudam,..."
1370678,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o,100.0,,"trabalhos, eu, os, carinho, aqui, muito, para,..."
1370679,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o,100.0,,"beijo, pela, próximo, todos, no, de, até, cora..."


In [22]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA,85.7,estevam,"olá, os, seguidores, do, à, todos"
1,1,52.520,53.500,é isso mesmo,ThC9MSlB1gA,100.0,NaN,"mesmo, é, isso"
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA,75.0,"ascendência, portuguesa","tem, pode, temas, diferentes, são, quem"
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA,90.0,permanência,"perguntas, em, à, portugal, mas, você, muitas,..."
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA,83.3,"nacionalidade, portuguesa","pessoas, vivendo, em, se, portugal, podem, mui..."
...,...,...,...,...,...,...,...,...
1370676,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o,94.1,NaN,"deixar, eu, mude, coisa, pode, nos, comentário..."
1370677,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,88.2,inscrevam,"receberem, canal, muito, no, se, para, ajudam,..."
1370678,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o,100.0,NaN,"trabalhos, eu, os, carinho, aqui, muito, para,..."
1370679,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o,100.0,NaN,"beijo, pela, próximo, todos, no, de, até, cora..."


In [23]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA,85.7,estevam,"olá, os, seguidores, do, à, todos"
1,1,52.520,53.500,é isso mesmo,ThC9MSlB1gA,100.0,NaN,"mesmo, é, isso"
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA,75.0,"ascendência, portuguesa","tem, pode, temas, diferentes, são, quem"
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA,90.0,permanência,"perguntas, em, à, portugal, mas, você, muitas,..."
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA,83.3,"nacionalidade, portuguesa","pessoas, vivendo, em, se, portugal, podem, mui..."
...,...,...,...,...,...,...,...,...
1360964,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o,94.1,NaN,"deixar, eu, mude, coisa, pode, nos, comentário..."
1360965,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,88.2,inscrevam,"receberem, canal, muito, no, se, para, ajudam,..."
1360966,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o,100.0,NaN,"trabalhos, eu, os, carinho, aqui, muito, para,..."
1360967,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o,100.0,NaN,"beijo, pela, próximo, todos, no, de, até, cora..."


In [24]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA,85.7,estevam,"olá, os, seguidores, do, à, todos",olá à todos os seguidores do
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA,75.0,"ascendência, portuguesa","tem, pode, temas, diferentes, são, quem",são temas diferentes quem tem
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA,90.0,permanência,"perguntas, em, à, portugal, mas, você, muitas,...",mas você recebeu muitas perguntas relacionadas à
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA,83.3,"nacionalidade, portuguesa","pessoas, vivendo, em, se, portugal, podem, mui...",muitas pessoas querem saber se vivendo em port...
5,5,97.080,98.820,portanto a resposta é sim,ThC9MSlB1gA,100.0,NaN,"a, é, portanto, sim, resposta",portanto a resposta é sim
...,...,...,...,...,...,...,...,...,...
1360964,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o,94.1,NaN,"deixar, eu, mude, coisa, pode, nos, comentário...",coisa que vocês não gostaram e queira que eu m...
1360965,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,88.2,inscrevam,"receberem, canal, muito, no, se, para, ajudam,...",vocês me ajudam muito fazendo isso e por favor se
1360966,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o,100.0,NaN,"trabalhos, eu, os, carinho, aqui, muito, para,...",os próximos trabalhos que eu vou fazendo aqui ...
1360967,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o,100.0,NaN,"beijo, pela, próximo, todos, no, de, até, cora...",um beijo no coração de todos obrigada pela ate...


In [25]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,olá à todos os seguidores do,9.740,11.125556,olá à todos os seguidores do estevam,ThC9MSlB1gA
1,são temas diferentes quem tem,82.480,84.269474,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA
2,mas você recebeu muitas perguntas relacionadas à,86.040,89.660556,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA
3,muitas pessoas querem saber se vivendo em port...,91.380,95.507586,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA
4,portanto a resposta é sim,97.080,98.820000,portanto a resposta é sim,ThC9MSlB1gA
...,...,...,...,...,...
1022127,coisa que vocês não gostaram e queira que eu m...,2399.640,2404.680000,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o
1022128,vocês me ajudam muito fazendo isso e por favor se,2404.680,2408.570625,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o
1022129,os próximos trabalhos que eu vou fazendo aqui ...,2412.150,2418.990000,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o
1022130,um beijo no coração de todos obrigada pela ate...,2418.990,2425.020000,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o


In [26]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,olá à todos os seguidores do,9.440,11.425556,olá à todos os seguidores do estevam,ThC9MSlB1gA
1,são temas diferentes quem tem,82.180,84.569474,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA
2,mas você recebeu muitas perguntas relacionadas à,85.740,89.960556,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA
3,muitas pessoas querem saber se vivendo em port...,91.080,95.807586,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA
4,portanto a resposta é sim,96.780,99.120000,portanto a resposta é sim,ThC9MSlB1gA
...,...,...,...,...,...
1022127,coisa que vocês não gostaram e queira que eu m...,2399.340,2404.980000,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o
1022128,vocês me ajudam muito fazendo isso e por favor se,2404.380,2408.870625,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o
1022129,os próximos trabalhos que eu vou fazendo aqui ...,2411.850,2419.290000,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o
1022130,um beijo no coração de todos obrigada pela ate...,2418.690,2425.320000,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o


In [27]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,olá à todos os seguidores do,9,11,olá à todos os seguidores do estevam,ThC9MSlB1gA
1,são temas diferentes quem tem,82,85,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA
2,mas você recebeu muitas perguntas relacionadas à,86,90,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA
3,muitas pessoas querem saber se vivendo em port...,91,96,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA
4,portanto a resposta é sim,97,99,portanto a resposta é sim,ThC9MSlB1gA
...,...,...,...,...,...
1022127,coisa que vocês não gostaram e queira que eu m...,2399,2405,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o
1022128,vocês me ajudam muito fazendo isso e por favor se,2404,2409,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o
1022129,os próximos trabalhos que eu vou fazendo aqui ...,2412,2419,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o
1022130,um beijo no coração de todos obrigada pela ate...,2419,2425,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o


In [28]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,olá uma das maiores dificuldades de quem começ...,0,1,olá uma das maiores dificuldades de quem começ...,ewjONSZfF14
1,em primeiro lugar o que pode ser feito com rel...,132,154,em primeiro lugar o que pode ser feito com rel...,7jbr0301cE4
2,e se você ainda não se convenceu de que esse a...,43,61,e se você ainda não se convenceu de que esse a...,urF2vcyozkg
3,por exemplo qual o limite de velocidade que o ...,149,180,por exemplo qual o limite de velocidade que o ...,6Eq7Tt8sz9M
4,na sua empresa precisa ter um documento que al...,107,123,na sua empresa precisa ter um documento que al...,6Eq7Tt8sz9M
...,...,...,...,...,...
1022127,w h o i s,247,251,no site whois w h o i s,cniV5tTzaVg
1022128,e o ó é a,527,529,é a penúltima e o ó é a antepenúltima então el...,AwkUylKTKPg
1022129,é o a o a,296,298,é o a o a,anYu8Sq4iBM
1022130,w s e e n,1091,1095,w s e e n,4do2NWAo5Gw


In [29]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,olá uma das maiores dificuldades de quem começ...,0,1,olá uma das maiores dificuldades de quem começ...,ewjONSZfF14,89
1,em primeiro lugar o que pode ser feito com rel...,132,154,em primeiro lugar o que pode ser feito com rel...,7jbr0301cE4,70
2,e se você ainda não se convenceu de que esse a...,43,61,e se você ainda não se convenceu de que esse a...,urF2vcyozkg,67
3,por exemplo qual o limite de velocidade que o ...,149,180,por exemplo qual o limite de velocidade que o ...,6Eq7Tt8sz9M,69
4,na sua empresa precisa ter um documento que al...,107,123,na sua empresa precisa ter um documento que al...,6Eq7Tt8sz9M,68
...,...,...,...,...,...,...
1022127,w h o i s,247,251,no site whois w h o i s,cniV5tTzaVg,100
1022128,e o ó é a,527,529,é a penúltima e o ó é a antepenúltima então el...,AwkUylKTKPg,100
1022129,é o a o a,296,298,é o a o a,anYu8Sq4iBM,60
1022130,w s e e n,1091,1095,w s e e n,4do2NWAo5Gw,80


In [30]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ
...,...,...,...,...,...
737044,é o mar e a,384,385,é o mar e a belíssima wakanda não faz fronteira,QIJEFBBuO2I
737045,c g am e f,627,630,c g am e f,ijjlCyli9tM
737046,x y z né e,4171,4174,carga horária x y z né e,RIdNWADMmic
737047,w h o i s,247,251,no site whois w h o i s,cniV5tTzaVg


In [31]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-31-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,100
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o,100
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,100
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE,100
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ,100
...,...,...,...,...,...,...
737044,é o mar e a,384,385,é o mar e a belíssima wakanda não faz fronteira,QIJEFBBuO2I,23
737045,c g am e f,627,630,c g am e f,ijjlCyli9tM,100
737046,x y z né e,4171,4174,carga horária x y z né e,RIdNWADMmic,41
737047,w h o i s,247,251,no site whois w h o i s,cniV5tTzaVg,39


In [32]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ
...,...,...,...,...,...
352110,a raça é e aí,480,490,a raça é e aí,QAOW1gfkxtA
352111,não é um e aí,367,379,não é um e aí,40g8pAyFlmo
352112,e aí o que é,610,660,e aí o que é,bgr5R2VyUO8
352113,o dia e de r,445,451,o dia e de r,TbC_JQMOjTM


In [33]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,nossos amiguinhos quando eles estão doentes,zzU2tAy9XmQ
1,ah queria tanto brincar lá fora,zzU2tAy9XmQ
2,meu amigo pensa numa briga difícil,zxA3ZZU2LJo
3,algum procedimento para diminuir esse volume v...,zwQndtG5Pps
4,vamos lá guardar nossas coisinhas pra vir busc...,zv8X54UYGNU
...,...,...
3029,quê carne maravilhosa vou comer esse daqui cor...,06yGYboeZFs
3030,vá recolher tudo até agora são quantas horas,02gCx_khxhI
3031,ordem das perguntas cada aluno irá,00UmvP5q7oA
3032,alinhar à direita posso alterar remover,00UmvP5q7oA


In [34]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ
...,...,...,...,...,...
2971,deve ter ali uns dez,18,20,deve ter ali uns dez,lAYoCa8atdM
2972,you just have to use,21,25,you just have to use,HlSdqW01_Ro
2973,red bull te dá asas,252,256,red bull te dá asas,WbXkzkjvagg
2974,i have it now with,323,329,i have it now with,PKWFlE01Dh8


In [35]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o,https://www.youtube.com/watch?v=BampggOg28o&t=...
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,https://www.youtube.com/watch?v=vwJq3pwexqc&t=...
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE,https://www.youtube.com/watch?v=mAHS9bghaLE&t=...
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ,https://www.youtube.com/watch?v=fvtbRqPsTtQ&t=40s
...,...,...,...,...,...,...
2971,deve ter ali uns dez,18,20,deve ter ali uns dez,lAYoCa8atdM,https://www.youtube.com/watch?v=lAYoCa8atdM&t=18s
2972,you just have to use,21,25,you just have to use,HlSdqW01_Ro,https://www.youtube.com/watch?v=HlSdqW01_Ro&t=21s
2973,red bull te dá asas,252,256,red bull te dá asas,WbXkzkjvagg,https://www.youtube.com/watch?v=WbXkzkjvagg&t=...
2974,i have it now with,323,329,i have it now with,PKWFlE01Dh8,https://www.youtube.com/watch?v=PKWFlE01Dh8&t=...


In [36]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [37]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [38]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [271]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Spanish_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [272]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [273]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=word_group_adjust_select, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
#df_word_group_time_loc_in_text_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Adjust Result Analysis Multiprocess

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = True  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to all word group length

# # shared word frequency
# shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list3 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list3.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list3), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_text_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
        
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [ ]:
df_adjust_text_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_text_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_text_ratio.reset_index(drop=True, inplace=True)
df_adjust_text_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_All_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000